In [21]:
import os
from dotenv import load_dotenv
import json 
from openai import OpenAI

In [22]:
load_dotenv()

True

In [23]:
XAI_API_KEY = os.getenv("XAI_API_KEY", "")
XAI_BASE_URL = os.getenv("XAI_BASE_URL", "")

In [24]:
client = OpenAI(
    base_url=XAI_BASE_URL,
    api_key=XAI_API_KEY
)

In [25]:
def get_current_temperature(location: str, unit: str = "fahrenheit"):
    temperature: int
    if unit.lower() == "fahrenheit":
        temperature = 59
    elif unit.lower() == "celsius":
        temperature = 15
    else:
        raise ValueError("unit must be one of fahrenheit or celsius")
    return {"location": location, "temperature": temperature, "unit": "fahrenheit"}


def get_current_ceiling(location: str):
    return {
        "location": location,
        "ceiling": 15000,
        "ceiling_type": "broken",
        "unit": "ft",
    }

tools_map = {
    "get_current_temperature": get_current_temperature,
    "get_current_ceiling": get_current_ceiling,
}


In [26]:
tools_definition = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature of the given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "default": "celsius"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_ceiling",
            "description": "Get the current cloud ceiling in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    }
                },
                "required": ["location"]
            }
        }
    }
]


In [35]:
messages = [{"role": "user", "content": "Hi, find me the height of the ceiling and temperature in Madagascar."}]
response = client.chat.completions.create(
    model="grok-2-1212",
    messages=messages,
    tools=tools_definition,
    tool_choice="auto",
    stream=True
)

fn_call_in_progress = False
fn_results = []

for chunk in response:

    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)


    if chunk.choices[0].delta.tool_calls:
        print("\n")
        
        for tool_call in chunk.choices[0].delta.tool_calls:

            fn_name = tool_call.function.name
            fn_args = json.loads(chunk.choices[0].delta.tool_calls[0].function.arguments)

        result = tools_map[fn_name](**fn_args)

        messages.append(
            {
                "role": "tool",
                "content": json.dumps(result),
                "tool_call_id": tool_call.id  # tool_call.id supplied in Grok's response
            }
        )

if messages[-1]['role'] == 'tool':
    response = client.chat.completions.create(
        model="grok-2-1212",
        messages=messages,
        tools=tools_definition,
        tool_choice="auto",
        stream=True
    )

    for chunk in response:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="", flush=True)  

I can find the current temperature in Madagascar, but for the cloud ceiling height, I need the specific city and state. Could you please provide that?